In [ ]:
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np  
import random


DEBUG_MODE = False

json_file = "/home/fpgadeveloper/Desktop/100/T20.json"  # Input JSON file
output_file = "../Application/T20.json"  # Output JSON file
#output_file1 = "c/TNC5000.json"  # Output JSON file

In [21]:

def read_application_model():
    with open(json_file) as f:
        app_model = json.load(f)
    return app_model


In [22]:
def find_sender_receiver_pairs(AM):
    sender_receiver_pair = {}
    for sd_re in AM["application"]["messages"]:
        sender = sd_re["sender"]
        receiver = sd_re["receiver"]
        if sender not in sender_receiver_pair:
            sender_receiver_pair[sender] = [receiver]
        else:
            sender_receiver_pair[sender].append(receiver)
    return sender_receiver_pair



# Function to find the sender receiver pairs as tuples , this for plotting the graph at later stages
def find_sender_receiver_pairs_tuple(sender_receiver_pair):
    vertex_edge_pairs = []

    for source, targets in sender_receiver_pair.items():
        for target in targets:
            vertex_edge_pairs.append((source, target))
    return vertex_edge_pairs

In [23]:
def plot_graph(vertex_edge_pairs):
    G = nx.DiGraph()
    for v,e in vertex_edge_pairs:
        G.add_edge(v,e)
   
    pos = nx.spring_layout(G, k=30)
    plt.figure(figsize=(18, 15))
    if DEBUG_MODE:
        nx.draw(G, with_labels=True, node_size=500, node_color='lightblue', font_size=10, font_color='black', arrowsize=10)
        plt.show()
    return G

In [24]:
AM = read_application_model()
sender_receiver_pair = find_sender_receiver_pairs(AM)
vertex_edge_pairs = find_sender_receiver_pairs_tuple(sender_receiver_pair)
G = plot_graph(vertex_edge_pairs)
print(G)

DiGraph with 20 nodes and 24 edges


<Figure size 1800x1500 with 0 Axes>

In [25]:


def finding_the_StartEnd_node(graph):
    # Find starting nodes (nodes with no incoming edges)
    starting_nodes = [node for node in graph.nodes() if graph.in_degree(node) == 0]

    # Find end nodes (nodes with no outgoing edges)
    end_nodes = [node for node in graph.nodes() if graph.out_degree(node) == 0]

    return starting_nodes, end_nodes

starting_nodes, end_nodes = finding_the_StartEnd_node(G)

print("Starting nodes: ", starting_nodes)
print("End nodes: ", end_nodes)

Starting nodes:  [5, 11, 16, 18, 19]
End nodes:  [0, 1, 2, 6, 13]


In [26]:
can_run_on_starting_and_end_nodes = {1:{1},2:{2},3:{3},4:{4},5:{1,3,4},6:{3,4}}
can_run_on_other_nodes = {1:{1},2:{1,2},3:{1,5},4:{1,6},5:{2},6:{1,5,6}}

In [27]:


def update_can_run_on(json_data, stratnodes, Endnodes,can_run_on_starting_and_end_nodes,can_run_on_other_nodes ):
    """
    Updates the `can_run_on` field in the JSON data to a specified list for all jobs.

    Args:
        json_data (dict): The original JSON data as a Python dictionary.
        new_can_run_on (list): The new list of values for the `can_run_on` field.

    Returns:
        dict: The updated JSON data.
    """
    for job in json_data.get("application", {}).get("jobs", []):
        if job["id"] in stratnodes or job["id"] in Endnodes:    
            job["can_run_on"] = list(can_run_on_starting_and_end_nodes[random.choice(list(can_run_on_starting_and_end_nodes.keys()))])    
        else:
            job["can_run_on"] = list(can_run_on_other_nodes[random.choice(list(can_run_on_other_nodes.keys()))] ) 
    return json_data

def save_json_to_file(json_data, file_name):
    """
    Saves the JSON data to a file.

    Args:
        json_data (dict): The JSON data to save.
        file_name (str): The name of the file to save the data to.
    """
    with open(file_name, "w") as file:
        json.dump(json_data, file, indent=4)

# Example usage

# Load JSON data from the input file
with open(json_file, "r") as file:
    data = json.load(file)

# Update `can_run_on` field
updated_data = update_can_run_on(AM, starting_nodes, end_nodes, can_run_on_starting_and_end_nodes,can_run_on_other_nodes)

print("Updated JSON:", updated_data)

# # Save the updated JSON to the output file
save_json_to_file(updated_data, output_file)

print(f"Updated JSON saved to {output_file}")


Updated JSON: {'application': {'jobs': [{'id': 0, 'wcet_fullspeed': 88, 'mcet': 0, 'deadline': 10000, 'can_run_on': [2], 'processing_times': 10}, {'id': 1, 'wcet_fullspeed': 17, 'mcet': 0, 'deadline': 10000, 'can_run_on': [2], 'processing_times': 5}, {'id': 2, 'wcet_fullspeed': 45, 'mcet': 0, 'deadline': 10000, 'can_run_on': [1], 'processing_times': 10}, {'id': 3, 'wcet_fullspeed': 8, 'mcet': 0, 'deadline': 10000, 'can_run_on': [1, 6], 'processing_times': 5}, {'id': 4, 'wcet_fullspeed': 18, 'mcet': 0, 'deadline': 10000, 'can_run_on': [1], 'processing_times': 1}, {'id': 5, 'wcet_fullspeed': 4, 'mcet': 0, 'deadline': 10000, 'can_run_on': [3], 'processing_times': 2}, {'id': 6, 'wcet_fullspeed': 16, 'mcet': 0, 'deadline': 10000, 'can_run_on': [4], 'processing_times': 2}, {'id': 7, 'wcet_fullspeed': 51, 'mcet': 0, 'deadline': 10000, 'can_run_on': [1, 6], 'processing_times': 4}, {'id': 8, 'wcet_fullspeed': 82, 'mcet': 0, 'deadline': 10000, 'can_run_on': [1, 6], 'processing_times': 10}, {'id'

In [133]:

### For Non Constrain Json File

import json

def update_can_run_on(json_data, new_can_run_on):
    """
    Updates the `can_run_on` field in the JSON data to a specified list for all jobs.

    Args:
        json_data (dict): The original JSON data as a Python dictionary.
        new_can_run_on (list): The new list of values for the `can_run_on` field.

    Returns:
        dict: The updated JSON data.
    """
    for job in json_data.get("application", {}).get("jobs", []):
        job["can_run_on"] = new_can_run_on
    return json_data

def save_json_to_file(json_data, file_name):
    """
    Saves the JSON data to a file.

    Args:
        json_data (dict): The JSON data to save.
        file_name (str): The name of the file to save the data to.
    """
    with open(file_name, "w") as file:
        json.dump(json_data, file, indent=4)


# Load JSON data from the input file
with open(json_file, "r") as file:
    data = json.load(file)

# Update `can_run_on` field
updated_data = update_can_run_on(data, [1, 2, 3, 4, 5, 6])

# Save the updated JSON to the output file
save_json_to_file(updated_data, output_file1)

print(f"Updated JSON saved to {output_file1}")


Updated JSON saved to c/TNC5000.json
